# PyTorch - podstawy

### Utworzenie środowiska wirtualnego i instalacja wymaganych modułów

Otwórz Terminal w tym samym folderze, w którym znajduje się skrypt:
- Utwórz środowisko wirtualne w folderze .env

```python -m venv .env```
- Aktywuj środowisko

```.env\Source\activate```

- Zaktualizuj PIP

```python -m pip install --upgrade pip setuptools wheel```
- Instaluj wymagane moduły

```pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126```

- Pamiętaj o wybraniu odpowiedniego kernela (kernela z utworzonego .env) w Visual Studio Code!

### Wykrycie i użycie odpowiedniego hardware'u

In [1]:
import torch
import torch.backends.mps
import importlib

# Sprawdzenie dostępności CUDA (NVIDIA GPU)
cuda_available = torch.cuda.is_available()

# Sprawdzenie liczby dostępnych GPU
cuda_device_count = torch.cuda.device_count() if cuda_available else 0

# Sprawdzenie dostępności MPS (Apple Silicon - M1/M2/M3)
mps_available = torch.backends.mps.is_available() if importlib.util.find_spec("torch.backends.mps") else False

# Sprawdzenie dostępności ROCm (AMD GPU)
rocm_available = torch.cuda.is_available() and torch.version.hip is not None

# Sprawdzenie CPU (PyTorch zawsze obsługuje CPU)
cpu_available = True

# Wyświetlenie wyników
print(f"CUDA (NVIDIA): {'Dostępne ✅' if cuda_available else 'Brak ❌'} ({cuda_device_count} GPU)")
print(f"MPS (Apple Silicon): {'Dostępne ✅' if mps_available else 'Brak ❌'}")
print(f"ROCm (AMD): {'Dostępne ✅' if rocm_available else 'Brak ❌'}")
print(f"CPU: {'Dostępne ✅' if cpu_available else 'Brak ❌'}")

CUDA (NVIDIA): Dostępne ✅ (1 GPU)
MPS (Apple Silicon): Brak ❌
ROCm (AMD): Brak ❌
CPU: Dostępne ✅


### Ustawienie zmiennej 'device' na wybrany hardware

In [2]:
device = 'cpu'
if cuda_available:
    device = 'cuda'
elif mps_available:
    device = 'mps'
elif rocm_available:
    device = 'rocm'

print(f'Wybrane urządzenie: {device} ✅')

Wybrane urządzenie: cuda ✅


### Operacje na tensorach

In [8]:
import torch

# Tensor z wartościami określonymi ręcznie
x = torch.tensor([[1, 2], [3, 4], [5, 6]])
y = torch.tensor([[6, 5], [4, 3], [2, 1]])
z = torch.tensor([[6, 5, 4], [3, 2, 1]])

# Dodawanie tensorów
sum_result = x + y
print("\nDodawanie tensorów:\n", sum_result)

# Mnożenie tensorów (element-wise)
mul_result = x * y
print("\nMnożenie tensorów (element-wise):\n", mul_result)

# Mnożenie macierzowe (dot product)
dot_result = torch.matmul(x, z)
print("\nMnożenie macierzowe:\n", dot_result)

# Transpozycja macierzy
transpose_result = x.T
print("\nTranspozycja macierzy:\n", transpose_result)


Dodawanie tensorów:
 tensor([[7, 7],
        [7, 7],
        [7, 7]])

Mnożenie tensorów (element-wise):
 tensor([[ 6, 10],
        [12, 12],
        [10,  6]])

Mnożenie macierzowe:
 tensor([[12,  9,  6],
        [30, 23, 16],
        [48, 37, 26]])

Transpozycja macierzy:
 tensor([[1, 3, 5],
        [2, 4, 6]])


### Automatyczne różniczkowanie (autograd)

In [ ]:
import torch

x = torch.tensor(2.0, requires_grad=True)
y = x ** 2 + 3*x        # Funkcja kwadratowa y(x) = x^2 + 3x
y.backward()            # Propagacja wsteczna, w tym liczenie gradientu
print(x.grad)           # Pochodna funkcji dy/dx = 2x + 3 dla x = 2 dy/dx = 7

tensor(7.)


In [ ]:
import torch

# Przykład aktualizacji wag modelu
# Załóżmy, że parametr (jedna z wag) modelu w = 5
w = torch.tensor(5.0, requires_grad=True)

# Funkcja błędu (jak błąd zależy od wag modelu): Loss = (w - 3)^2
loss = (w - 3) ** 2

# Obliczenie gradientu (aby wiedzieć jak zmiana wag wpłynie na błąd)
# dLoss/dw = 2*(w - 3) = 2*(5 - 3) = 4
loss.backward()     # <-- Wystarczy jedno zawołanie funkcji

# Aktualizacja parametru ręcznie (symulacja optymalizacji SGD)
learning_rate = 0.01
w.data -= learning_rate * w.grad    # 5 - 0.01 * 4 = 4.96

# Wyzerowanie gradientu przed kolejną iteracją
w.grad.zero_()

print(f"Nowa wartość w: {w:.2f}")

Nowa wartość w: 4.96


### Tworzenie modelu głębokiej sieci neuronowej

In [14]:
import torch.nn as nn

class DenseNeuralNetwork(nn.Module):
    def __init__(self):
        super(DenseNeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(10, 50)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(50, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

model = DenseNeuralNetwork().to(device)
print(model)

DenseNeuralNetwork(
  (layer1): Linear(in_features=10, out_features=50, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=50, out_features=1, bias=True)
)


### Tworzenie modelu głębokiej sieci neuronowej używając klasy **Sequential**

In [3]:
import torch.nn as nn

model = nn.Sequential(  # Warstwy sekwencyjne głębokiej sieci FCN, kolejno:
    nn.Linear(10, 50),  # warstwa liniowa (10 wejść -> 50 neuronów),
    nn.ReLU(),          # funkcja aktywacji 1-wszej warstwy,
    nn.Linear(50, 1)    # warstwa końcowa (50 -> 1 neuronów).
)

### Trening zdefiniowanego modelu

In [4]:
import torch.optim as optim

# Optymalizator (Adam) i funkcja kosztu (MSE)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# Dane wejściowe i oczekiwane wyniki (zazwyczaj przygotowane wcześniej jako dane rzeczywiste lub syntetyczne)
inputs = torch.randn(5, 10)  # 5 próbek po 10 cech każda
targets = torch.randn(5, 1)  # Oczekiwane wyniki

# Trening (1 epoka)
for epoch in range(1):
    optimizer.zero_grad()               # Zerowanie gradientów
    outputs = model(inputs)             # Przewidywanie (inferencja)
    loss = loss_fn(outputs, targets)    # Obliczenie błędu
    loss.backward()                     # Propagacja wsteczna błędu 
    optimizer.step()                    # Aktualizacja wag

    print(f"Strata: {loss.item():.3f}")

Strata: 1.257


### Definiowanie własnego zbioru danych do treningu sieci

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Przykładowy zbiór danych
class CustomDataset(Dataset):
    def __init__(self, size):
        self.data = torch.randn(size, 10)           # 100 losowych próbek, każda ma 10 cech
        self.labels = torch.randint(0, 2, (size,))  # Losowe etykiety 0 lub 1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Tworzymy instancję datasetu
dataset = CustomDataset(size=1000)

### Przygotowanie obiektu ładującego wcześniej przygotowane dane w postaci paczek (batch)

In [ ]:
# DataLoader: batch_size=32, shuffle=True, 12 wątków (równolegle) do ładowania danych
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=12)

### Fine tuning (dostrajanie) istniejącego modelu

In [ ]:
import torchvision.models as models

# Pobranie modelu ResNet50 wstępnie wytrenowanego na zbiorze ImageNet
model = models.resnet50(pretrained=True)

# Zamrażanie wag modelu
for param in model.parameters():
    param.requires_grad = False

# Zmiana ostatniej warstwy, aby model uczył się nowego zadania
# na podstawie nowego zbioru danych (np. zdjęć różnych silników spalinowych)
num_ftrs = model.fc.in_features     
model.fc = nn.Linear(num_ftrs, 10)  # 10 klas, np. rozpoznawanie 10 typów silników spalinowych

print("Gotowy do trenowania na nowych danych!")

# Dalej można realizować już standardowy trening głębokiego modelu
# ...